# New Section

In [1]:

import sys
import os

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import sklearn.model_selection
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

import datetime

import joblib
import pickle
import tempfile


import boto3
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()


# S3 bucket name
bucket = 'sagemaker-us-west-2-479862333671'
subfolder = ''
conn = boto3.client('s3')
contents = conn.list_objects(Bucket=bucket, Prefix=subfolder)['Contents']


In [2]:
model_data='s3://sagemaker-us-west-2-479862333671/perdictions/output/sagemaker-xgboost-2022-01-10-08-59-02-189/output/model.tar.gz'

In [3]:
model_data

's3://sagemaker-us-west-2-479862333671/perdictions/output/sagemaker-xgboost-2022-01-10-08-59-02-189/output/model.tar.gz'

In [4]:
from sagemaker import image_uris
image_uris.retrieve(framework='xgboost',region='us-west-2',version='1.2-1')

'246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost:1.2-1'

In [5]:
image=image_uris.retrieve(framework='xgboost',region='us-west-2',version='1.2-1')

In [6]:
image

'246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost:1.2-1'

In [7]:
role

'arn:aws:iam::479862333671:role/service-role/AmazonSageMaker-ExecutionRole-20220103T082598'

In [14]:
val = pd.read_csv('s3://{}/{}'.format(bucket, 'data/val.csv'))

In [15]:
val

,demand,item_id,dept_id,cat_id,store_id,state_id,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,date_day,date_week,date_month,date_year,date_week_m,lag_28,lag_35,lag_42,lag_49,lag_56,rolling_median_7,rolling_median_14,rolling_median_30,rolling_median_60,rolling_median_180
0,0,1437,3,1,0,0,1914,30,4,4,2,0,0,0,8.38,25,17,4,5,4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1437,3,1,0,0,1915,30,4,4,2,0,0,0,8.38,26,17,4,5,4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0,1437,3,1,0,0,1916,30,4,4,2,0,0,0,8.38,27,17,4,5,4,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,2,1437,3,1,0,0,1917,30,4,4,2,0,0,0,8.38,28,17,4,5,4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0,1437,3,1,0,0,1918,30,4,4,2,0,0,0,8.38,29,17,4,5,5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
823225,1,1432,2,0,9,2,1940,30,4,4,2,0,0,0,2.98,21,20,5,5,3,0.0,0.0,0.0,0.0,0.0,1.0,0.5,1.0,1.0,1.0
823226,1,1433,2,0,9,2,1940,30,4,4,2,0,0,0,2.48,21,20,5,5,3,3.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
823227,0,1434,2,0,9,2,1940,30,4,4,2,0,0,0,3.98,21,20,5,5,3,5.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0
823228,1,1435,2,0,9,2,1940,30,4,4,2,0,0,0,1.28,21,20,5,5,3,0.0,0.0,3.0,0.0,2.0,0.0,0.0,1.0,1.0,1.0


In [43]:
X_val = val.iloc[:,1:]

In [30]:
val.iloc[:,0].values

array([0, 0, 0, ..., 0, 1, 5])

In [45]:
data_dir = ''

X_val.to_csv(os.path.join(data_dir, 'X_val_values.csv'), index=False, header=False)

X_val_location = sagemaker_session.upload_data(os.path.join(data_dir, 'X_val_values.csv'))


In [54]:
X_val = pd.read_csv('s3://{}/{}'.format(bucket, 'data/X_val_values.csv')).to_numpy()

In [55]:
X_val

array([[1.437e+03, 3.000e+00, 1.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [1.437e+03, 3.000e+00, 1.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [1.437e+03, 3.000e+00, 1.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [1.434e+03, 2.000e+00, 0.000e+00, ..., 1.000e+00, 1.000e+00,
        1.000e+00],
       [1.435e+03, 2.000e+00, 0.000e+00, ..., 1.000e+00, 1.000e+00,
        1.000e+00],
       [1.436e+03, 2.000e+00, 0.000e+00, ..., 1.500e+00, 2.000e+00,
        2.000e+00]])

In [ ]:
trainedmodel = sagemaker.model.Model(
    image_uri=image,
    model_data = model_data,
    role=role)

In [39]:
model_transformer = trainedmodel.transformer(instance_count=1, instance_type='ml.c5.4xlarge')

In [ ]:
X_val_location = 's3://{}/{}'.format(bucket, 'data/X_val_values.csv')

In [49]:
model_transformer.transform(X_val_location, content_type='text/csv', split_type='Line')

.............................[2022-01-12:10:14:49:INFO] No GPUs detected (normal if no gpus installed)
[2022-01-12:10:14:49:INFO] No GPUs detected (normal if no gpus installed)
[2022-01-12:10:14:49:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|invocations|execution-parameters) {
      proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
      proxy_set_header Host $http_host;
      proxy_redirect off;
      proxy_read_timeout 60s;
      proxy_pass http://gunicorn;
    }
    location / {
      return 404 "{}";
    }
  }
}
[2022-01-12 10:14:49 +0000] [45] [INFO] Starting gunico

UnexpectedStatusException: Error for Transform job sagemaker-xgboost-2022-01-12-10-10-07-166: Failed. Reason: ClientError: See job logs for more information

In [ ]:
model_transformer.wait()

In [ ]:
!aws s3 cp --recursive $model_transformer.output_path $data_dir

In [ ]:
Y_pred = pd.read_csv(os.path.join(data_dir, 'X_val_values.csv.out'), header=None)

In [ ]:
Y_pred

In [ ]:
plt.scatter(Y_test, Y_pred)
plt.xlabel("Actual Demand")
plt.ylabel("Predicted Demand")
plt.title("Actual Demand vs Predicted Demand")
